# Transformée de Fourier à court terme (dite aussi à fenêtres).

In [ ]:
from __future__ import division
import holoviews as hv
hv.extension('bokeh')
import param
import holoviews as hv,panel as pn,param
from holoviews.streams import Pipe
import time
import pandas as pd
import panel as pn
from panel.pane import LaTeX
from scipy.io.wavfile import read
from IPython.display import Audio
import requests
from io import BytesIO
from PIL import Image
import shutil
from urllib.request import urlopen
import io
import scipy.io as sio

import numpy as np
import scipy as scp
import pylab as pyl
from matplotlib import cm
import matplotlib.pyplot as plt
from scipy import fftpack
from matplotlib.pyplot import imshow as imageplot
from mpl_toolkits.mplot3d import Axes3D
import wave
import warnings
warnings.filterwarnings('ignore')

In [ ]:
Saxo=sio.loadmat('Saxo.mat')
Saxo1=Saxo['S']
Saxo2=Saxo['S1']
temp=np.zeros(913)
for k in range(0,913):
    temp[k]=Saxo2[k]
Saxo2=temp
N1=int(len(Saxo1)/3)
temp=np.zeros(N1)
for k in range(0,N1):
    temp[k]=Saxo1[k]
sax=temp
Fe=44100
SonGuitare="Guitare.wav"
gui=read(SonGuitare)
guit=gui[1]/2**14

## Introduction.

L'analyse de Fourier consiste à décrire un signal ou un son comme une somme de sinusoides, discrètes ou continue. La commande FFT (Fast Fourier Transform) réalisée par la commande fftpack.fft en python permet de calculer les coefficients de décomposition d'un signal dans une telle base. 

Est-il pour autant toujours pertinent de décomposer un signal ou un son dans une telle base ? 

La réponse est : Non, pas toujours. Cette décomposition n'est utile, permettra de manipuler le son de manière efficace que si le son est stationnaire. On peut définir mathématiquement le concept de stationnarité mais pour faire simple, un signal est stationnaire s'il a un comportememnt invariant dans le temps. 

Prenons le son de saxo suivant :

In [ ]:
Audio(sax, rate=Fe)

In [ ]:
options = dict(width=800,height=300,xaxis=None,yaxis=None)
hv.Curve(sax).opts(**options)

En dehors du début de l'enregristrement, quel que soit l'endroit où vous zoomer, vous observerez le même motif, la même structure. On dit que le don est stationnaire. 

Regardons maintenant le son de guitare :

In [ ]:
Audio(guit, rate=Fe)

In [ ]:
p1=40000
p2=100000
l=2000
Extrait1=guit[p1:p1+l]
t1=np.arange(p1,p1+l)
Extrait2=guit[p2:p2+l]
t2=np.arange(p2,p2+l)
hv.Curve(guit).opts(width=800)*hv.Curve((t1,Extrait1)).opts(color='red')*hv.Curve((t2,Extrait2)).opts(color='green')

Si on zoom sur deux parties différentes, on n'obtient pas exactement la même chose, ceci est dû au fait que la guitare ne tient pas toujours la même note.

In [ ]:
pn.Column(hv.Curve(Extrait1).opts(width=800,color='red'),hv.Curve(Extrait2).opts(width=800,color='green'))

On dit que le son de guitare n'est pas stationnaire. Une analyse de Fourier sur l'intégralité du son de guitare, n'apporterai pas grand chose. Toutes les fréquences seraient mélangés. En traitement du son, l'immense majoriét des sons ne sont pas stationnaires. C'est pourquoi on effecrue un traitement par morceaux, par tranche, on dira plutôt par fenêtre. L'idée est de découper le son sur des intervalles où il est stationnaire et de calculer la transformée de Fourier sur chacune de ces tranches. 

On estime que les sons courants sont stationnaires sur des intervalles allant de 20 à 50 ms. Pour un son échantillonné à 44100Hz ça signifie qu'on calcule des transformée de Fourier sur environ 
$\frac{44100\times20}{1000}\approx 880$ et $\frac{44100\times50}{1000}\approx 2200$ points.

Comme la FFT se calcule rapidement pour des longueurs de vecteurs qui sont des puissances de 2 (algorithme encore plus rapide dans ce cas), je vais opter pour la suite pour des "fenêtres" de 1024 points.

## Une première TFCT élémentaire, un premier spetrogramme.

Une Transformée de Fourier à court terme (TFCT) est un tableau de nombres complexes dont chaque colonne contient la transformée de Fourier des différentes fenêtres du signal. Chaque colonne correspond à une fenêtre temporelle, chaque ligne à une "fréquence". On ne peut afficher un tableau complexe, la plupart du temps on affiche le module de cette transformée que l'on appelle aussi spectrogramme.

In [ ]:
def TFCT1(Son,N):
    NS=len(Son)
    Nf=int(np.floor(NS/N))
    TF=np.zeros((N,Nf),dtype=complex)
    for k in range(0,Nf):
        d=k*N
        f=d+N
        TF[:,k]=fftpack.fft(Son[d:f])
    return TF

On calcule ici le spectrogramme du son de saxo :

In [ ]:
N=1024
options2= dict(width=800,height=300,xaxis=None,yaxis=None)
TFsax=TFCT1(sax,N)
Spectresax=np.abs(TFsax)
hv.Image(Spectresax).opts(**options2)

Ici celui du son de guitare :

In [ ]:
N=1024
options2= dict(width=800,height=300,xaxis=None,yaxis=None)
TFguit=TFCT1(guit,N)
Spectreguit=np.abs(TFguit)
hv.Image(Spectreguit).opts(**options2)

On peut remarquer qu'on ne voit pas grand chose... et c'est normal si on affiche la totalité du spectrogramme. 
    Il faut avoir en tête que le son a été échantillonné à 44100Hz et qu'on effectue une analyse de Fourier sur 1024 points. Donc la première ligne du spectrogramme correspond en gros à la valeur moyenne du signal sonore sur chacune des fenêtres c'est à dire approximativement 0. La première ligne correspond à des Fréquences de l'ordre de $\frac{44100}{1024}\approx 43 Hz$, la deuxième ligne à des fréquences de l'ordre de 86Hz... la 21ème ligne à des fréquences de l'odre de 860Hz... autrement dit, une très grande partie de l'information de ce spectrogramme est située sur les toutes premières lignes... Pour y voir quelque chose il est préférable de zoomer, par exemple sur les 30 première lignes, où on visulise mieux le son de guitare...

In [ ]:
Spectreguitpartiel=Spectreguit[0:30,:]
hv.Raster(Spectreguitpartiel).opts(width=800,height=300)

In [ ]:
Audio(guit, rate=Fe)

Sur le son de Guitare, on peut observer qu'à la fin du son, la note tenue correspond à un coefficient de Fourier important sur la ligne d'indice 10. La fréquence jouée est docn approximativement $10\times 43=430$Hz. Il s'agit d'un La... Comme on fait une analyse que seulement 1024 points, la précision qu'on peut espérer n'est que de 43 hz. Si on veut être plus précis en fréquence, on peut faire une analyse de Fourier sur plus de points, mais on sera moins précis en temps. Vous pouvez le tester...

Si on zoom de la même façon sur le son de saxo, on va voir que les colonnes sont presques identiques, ceci traduit la stationnarité du son. De fait pour ce son particulier une analyse par fenêtre n'est pas la plus pertinente.


In [ ]:
hv.Image(Spectresax[0:30,:]).opts(**options2)

In [ ]:

def FTDyn(x,y,sound,TF):
    N,Nf=np.shape(TF)
    xx=int(x)
    yy=int(y)
    t=np.arange(xx*N,xx*N+N)
    fsin=np.zeros(N)
    fsin[yy]=1
    sin=200*np.real(fftpack.ifft(fsin))
    return hv.Curve(sound[xx*N:(xx+1)*N],kdims='time',vdims='pressure')\
.opts(width=800,height=200).redim.range(pressure=(-1,1))*hv.Curve(sin).opts(color='green')

def SegDyn(x,y,sound):
    xx=int(x)
    yy=int(y)
    t=np.arange(xx*N,xx*N+N)
    C1=hv.Curve(sound,kdims='time',vdims='pressure').opts(width=800,height=200).redim.range(pressure=(-1,1))
    C2=hv.Curve((t,sound[xx*N:xx*N+N]),kdims='time',vdims='pressure')\
    .opts(width=800,height=200).redim.range(pressure=(-1,1))
    return C1*C2

source1 = hv.Raster(Spectreguitpartiel,kdims = ['timewindow','freq']).opts(width=800,height=200)
pointer = hv.streams.PointerXY(source=source1,x=8,y=20)
#temp1=hv.DynamicMap(lambda x,y: hv.Points((x,y)).opts(size=20), streams=[pointer])
temp=hv.DynamicMap(lambda x,y: FTDyn(x,y,guit,TFguit), streams=[pointer])
temp2=hv.DynamicMap(lambda x,y: SegDyn(x,y,guit), streams=[pointer])
pn.Column(source1,temp2,temp)

Dans la figure précédente, en haut se trouve le spectrogramme, au milieu le son de guitare, en bas, l'extrait de son associé à la fenêtre que vous pointez sur le spectrogramme. En rouge sur le graphique du milieu est représenté l'extrait de son associé à la colonne du spectrogramme pointé par la souris. En vert sur la figure du dessous, la fréquence associé à la ligne pointé. En haut du spectrogramme, les basses fréquences, en bas les hautes fréquences. L'amplitude de la sinusoide verte, n'est pas lié au coefficient de Fourier, elle est juset constante. Le but est simplement que vous arriviez à comprendre ce que contient le spectrogramme.
L'intensité lumineuse correspond à l'amplitude du coefficient de Fourier associé.

## Nécessité d'un fenêtrage et d'un recouvrement.

Dans le version du spectrogramme qui a été présentée plus haut, nous avons segmenté le son sur des fenêtres disjointes. Schématiquement on a coupé violamment le signal en tranches : dans la figure suivante nous représentons deux tranches ou fenêtres d'analyse, chacune 

In [ ]:
Extrait3=guit[80000:80000+20000]
f1=np.zeros(20000)
f1[10000:11024]=1
d=11024
f2=np.zeros(20000)
f2[d:d+1024]=1
f3=f1*0
f3[10000:11024]=Extrait3[10000:11024]
f4=f1*0
f4[d:d+1024]=Extrait3[d:d+1024]
pn.Column(hv.Curve(Extrait3,kdims='time',vdims='pressure')\
.opts(width=800)*hv.Curve(f3)*hv.Curve(f4).opts(color='black')\
          *hv.Curve(f1).opts(color='red')*hv.Curve(f2).opts(color='black'),\
         hv.Curve(f3[10000:11024],kdims='time',vdims='pressure').opts(width=800,color='red',title='Fenêtre rouge')\
         ,hv.Curve(f4[d:d+1024],kdims='time',vdims='pressure').opts(width=800,color='black',title='Fenêtre noire'))

Le problème d'un tel découpage est qu'il altère fortement le contenu fréquenciel du signal. Si vous zoomez sur les basses fréquences vous verrez que sur l'extrait de guitare long, (20000 points), les pics fréquenciels sont beaucoup mieux marqués et plus propres. Le fait d'avoir coupé brutalement le signal a un peu modifié son contenu fréquentiel. Sans rentrer dans des explications techniques, le problème vient du fait qu'extraire un signal de cette façon revient à le multiplier par une fenêtre rectangulaire (la rouge et la noire sur le diagramme plus haut). Le fait que ces fenêtres sont discontinues aux bords altère le contenu fréquentiel du signal. 

In [ ]:
fguit_long=fftpack.fft(Extrait3)
fguit_short1=fftpack.fft(Extrait3[10000:11024])
pn.Column(hv.Curve(np.abs(fguit_long),kdims='frequences',vdims='intensity').opts(width=800,title='FFT de l extrait long'),
hv.Curve(np.abs(fguit_short1),kdims='freq',vdims='intensity').opts(width=800,title='FFT de la fenêtre rouge'))

Si on veut se servir du spectrogramme pour effectuer des traitement sur le son, comme le vocoder ou la séparation de sources, il est fortement conseillé de "prélever" les petits extraits de son en appliquant ce qu'on appelle un fenêtrage, c'est-à-dire en multipliant le morceaux de son par une "fenêtre" d'analyse qui va fortement atténuer le signa sur les bords. Le but étant d'altérer le contenu fréquentiel le moins possible :
https://en.wikipedia.org/wiki/Window_function
Nous utiliserons des fenêtres de Hanning  qui est juste une fonction $\cos^2(x)$ :

In [ ]:
H=np.hanning(1024)
hv.Curve(H).opts(width=800,title='Fenêtre de Hanning')

Ainsi avant de calculer les FFT de chacun des petits extraits de son sur 1024 points on va les multiplier par une fenêtre de Hanning qui va les écraser sur les bords.

In [ ]:
Extrait4=Extrait3[10000:11024]
hv.Curve(Extrait4,label='Extrait sonore').opts(width=900,height=350)*hv.Curve(H,label='Fenêtre de Hanning')\
*hv.Curve(H*Extrait4,label='Signal fenêtré').opts(color='black')

Un tel fenêtrage a un inconvénient majeur : on perd le son au niveau du bord des fenêtres. Ici on multimlisrait le son original par une succession de fenêtre de Hanning, ce qui impliquerait d'appliquer des transformée de Fourier local au signal représenté en noir, à la la place du bleu. Zoomez pour mieux voir ce qu'il se passe.

In [ ]:
def FenetrageBrutalHanning(son,n):
    NS=len(son)
    Nf=int(np.floor(NS/N))
    h=np.hanning(n)
    sf=0*son
    for k in np.arange(0,Nf):
        sf[k*N:(k+1)*N]=son[k*N:(k+1)*N]*h
    return sf
sf=FenetrageBrutalHanning(Extrait3,1024)
t=np.ones(20000)
hs=FenetrageBrutalHanning(t,1024)
hv.Curve(Extrait3)*hv.Curve(sf).opts(width=900,color='black')*hv.Curve(hs).opts(color='red')

Pour éviter cette perte d'information, la solution la plus couramment utilisée est de construire la TFCT avec des fenêtres qui se recouvrent. C'est-à-dire 

In [ ]:
def TFCourtTerme(Son,N,rec):
    H=np.hanning(N)
    NS=len(Son)
    Nf=int(np.floor(rec*NS/N)-rec+1)
    TF=np.zeros((N,Nf),dtype=complex)
    for k in range(0,Nf):
        d=int(k*N/rec)
        f=d+N
        TF[:,k]=fftpack.fft(Son[d:f]*H)
    return TF

In [ ]:
d=100000
ExtraitGuitare=guit[d:d+80*1024]
TF=TFCourtTerme(ExtraitGuitare,1024,2)
hv.Raster(np.abs(TF[0:50,:]))
hv.Curve(ExtraitGuitare).opts(width=650)

In [ ]:
N=1024
d=int(4*N/2)
f=d+N
H=np.hanning(N)
son=ExtraitGuitare[d:f]
sonetendu=ExtraitGuitare[d-N:f+N]
t=np.arange(d,f)
t1=np.arange(0,N)
t2=np.arange(N,2*N)
hv.Curve(sonetendu).opts(width=650,height=150)\
*hv.Curve((t2,H)).opts(color='red')*hv.Curve((t2,son*H)).opts(color='black')

In [ ]:
class ConstructionTfCT(param.Parameterized):
    k= param.Integer(10,bounds=(1,158))
    def view(self):
        N=1024
        d=int(self.k*N/2)
        f=d+N
        H=np.hanning(N)
        son=ExtraitGuitare[d:f]
        sonetendu=ExtraitGuitare[d-N:f+N]
        t=np.arange(d,f)
        t2=np.arange(N,2*N)
        TFp=np.abs(TF[0:50,:])
        TFp[:,self.k]=200
        C1=hv.Curve((t2-N/2,H)).opts(color='green')
        C2=hv.Curve((t2+N/2,H)).opts(color='green')
        C3=hv.Curve((t2+N,H)).opts(color='green')
        C4=hv.Curve((t2-N,H)).opts(color='green')
        CT=C1*C2*C3*C4
        return pn.Column(hv.Curve(ExtraitGuitare).opts(width=650,height=150)*hv.Curve((t,son)).opts(color='black'),\
                        hv.Curve(sonetendu).opts(width=650,height=150)\
                        *hv.Curve((t2,H)).opts(color='red')*hv.Curve((t2,son*H)).opts(color='black')*CT\
                         ,hv.Raster(TFp).opts(width=650,height=150))

On construit ainsi chaque colonne en multipliant le signal par une fenêtre de Hanning. J'ai indiqué en vert les fenêtres voisines, ici avec un recouvrement de moitié. La colonne du spectogramme en blanc dans le spectrogramme est la transformée de Fourier du signal noir sur la fenêtre du milieu. Ce signal noir est la multiplication du signal original indiqué en noir sur la figure du haut, multiplié par le fenêtre de Hanning en rouge. Le signal bleu dans la figure du milieu est un zoom sur la zone noire de la figure du haut. On peut se déplacer dans le signal avec le curseur à gauche.

In [ ]:
cons= ConstructionTfCT()
pn.Row(cons.param,cons.view)

Tous les choix de recouvrement sont possibles. On peut même décaler chaque fenêtre d'un point par rapport à la précédente. Et si notre objectif est d'analyser très finement un signal, il peut être pertinent de prendre un recouvrement très élevé. Notre objectif dans la suite est de reconstruire le signal à partir d'une TFCT qu'on aura évetnuellement modifiée pour effectuer des traitements, pour réaliser un vocoder puis pfaire de la séparation de sources. Les différents tests que j'ai pu faire indiquent qu'un recouvrement non pas d'une demi fenêtre mais de $\frac{7}{8}$ de fenêtre, c'est-à-dire un décalage d'un huitième de fenêtre entre deux colonnes consécutives du spectrogramme est un bon choix pour de telles applications, c'est pourquoi c'est le choix que nous ferons par la suite.


In [ ]:
TF2=TFCourtTerme(guit,1024,8)
Spec2=np.abs(TF2)
hv.Raster(Spec2[0:50,:]).opts(width=900)

## Reconstruction d'un son à partir d'une TFCT.

Un des intérêt de la TFCT est qu'elle permet la reconctruction du signal original. S'il n'y avait ni recouvrement, ni fenêtrage de Hannning, l'algorithme de reconstruction serait très simple. Il suffirait de calculer les FFT inverses des colonnes de la TFCT et de les mettre bout à bout. 

In [ ]:
def SyntheseElementaire(TF):
    N=np.shape(TF)[0]
    Nf=np.shape(TF)[1]
    Son=np.zeros(N*Nf)
    for k in range(0,Nf):
        d=int(k*N)
        f=d+N
        Son[d:f]=np.real(fftpack.ifft(TF[:,k]))
    return Son

In [ ]:
TF1=TFCT1(guit,1024)
Sonrec=SyntheseElementaire(TF1)
pn.Column(hv.Curve(Sonrec).opts(width=900,title='Son guitare original')\
         ,hv.Curve(Sonrec).opts(width=900,\
                                title='Son guitare reconstruit à partir d une TFCT sans recouvrement ni fenêtrage'))

Tant qu'on ne modifie pas la TFCT, cette méthide de reconstruction ne pose aucun problème. Pour autant, dans la pratique, on ne procède jamais ainsi. En effet recoller des petits bouts de sons traités independamment, les uns au bout des autres crée des effets sonores très désagréable dès que le recollement n'est pas parfait car on introduit des discontinuités sur le signal. Ces discontinuités, même petites induisent des hautes fréquence qui sont audibles et très désagréables. Pour palier ce problème, on ne met pas les sons élémentaires obtenus par transformée de Fourier inverses des colonnes bout à bout, on les multiplie par des fenêtres de Hanning et on les ajoute en les décalant selon le même décalage utilisé pour la construction de la TFCT. 

In [ ]:
N=1024
TF2=TFCourtTerme(guit,N,4)
TF1=TFCT1(guit,N)

In [ ]:
class Synthecompar(param.Parameterized):
    k1= param.Integer(1,bounds=(1,8))
    k2= param.Integer(1,bounds=(1,29))
    def view(self):
        d1=35
        d2=4*d1
        N=1024
        H=np.hanning(N)
        t=np.arange(0,N)
        sonel=np.real(fftpack.ifft(TF1[:,d1]))
        sonel2=np.real(fftpack.ifft(TF2[:,d2]))*H
        C1=hv.Curve(sonel).opts(width=650,height=150,yaxis=None,title='Son recosntruit sans recouvrement ni fenêtrage')
        C2=hv.Curve(sonel2).opts(width=650,height=150,yaxis=None,title='Chaque nouvelle trame est multiplié par une fenêtre de Hanning')
        sonref=np.zeros(N*8)
        sonref[0:N]=sonel
        for k in np.arange(1,self.k1):
            sonel=np.real(fftpack.ifft(TF1[:,d1+k]))
            C1=C1*hv.Curve((t+k*N,sonel)).opts(width=650)
        for j in np.arange(1,self.k2):
            sonel2=np.real(fftpack.ifft(TF2[:,d2+j]))
            C2=C2*hv.Curve((t+int(j*N/4),sonel2)).opts(width=650)
            d=int(j*N/4)
            sonref[d:d+N]=sonref[d:d+N]+sonel2
        C3=hv.Curve(sonref).opts(width=650,height=150,yaxis=None,title='Son reconstruit avec recouvrement et fenêtrage en ajoutant les trames fenêtrée')
        return pn.Column(C1,C2,C3)

In [ ]:
synt= Synthecompar()
pn.Row(synt.param,synt.view)


Vous pourrez noter qu'on reconstruit le signal parfaitement (sauf sur les bords). Le fait que la reconstruction n'est pas parfaite sur le bords n'est pas un problème car il s'agit de quelques centièmes de seconde au total.

Pourtant on a multiplié par une fenêtre de Hanning deux fois 
\begin{enumerate}
\item avant de caluler la FFT lors du calcul de la TFCT 
\item et après lors de la reconstruction du son.  
\end{enumerate}
Ici on n'a pas touché la TFCT donc chaque fenêtre a été multiplié par un $\cos^4(x)$ entre le signal original et le signal recosntruit. Pourtant il n'y a aucune distorsion sur le signal en dehors des bords.

Question : Expliquez ce phénomène ? 
Question : Proposez une fonction qui permet de reconstruire le signal à partir de la TFCT calculée avec recouvrement et fenêtrage.